In [ ]:
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta
from cs2_trading.data.inventory import Inventory
from cs2_trading.data.api import InfoAPI
from cs2_trading.agents.NewsAgent import NewsAgent
from cs2_trading.strategy import DailyStrategy

# 1. Setup Environment
load_dotenv()

# Initialize API clients
# Ensure you have GEMINI_API_KEY and INFO_API_TOKEN in .env
info_api = InfoAPI(os.getenv("INFO_API_TOKEN"))
news_agent = NewsAgent(llm_model="gemini-3-pro-preview")

# 2. Initialize Inventory
# Start with a fresh inventory or load existing
inventory_path = "cs2_trading/res/simulation_inventory.json"
if os.path.exists(inventory_path):
    os.remove(inventory_path) # Start fresh for simulation

my_inventory = Inventory()
# Add some initial items to start with
start_date = datetime.now() - timedelta(days=14) # Start simulation 14 days ago

my_inventory.add_item(
    id=11111, 
    name="Titan Holo Katowice 2014", 
    price=50000.0, 
    date=start_date - timedelta(days=10), # Bought even earlier
    info={"rarity": "Contraband"}
)
my_inventory.add_item(
    id=22222, 
    name="iBUYPOWER Holo Katowice 2014", 
    price=45000.0, 
    date=start_date, 
    info={"rarity": "Contraband"}
)

print("Initial Inventory:")
print(my_inventory)

# 3. Initialize Strategy
strategy = DailyStrategy(my_inventory, news_agent, info_api, llm_model="gemini-3-pro-preview")
strategy.target_quantity = 5 # Maintain 5 items

# 4. Run 14-Day Simulation
print("\n=== Starting 14-Day Simulation ===")

for day in range(14):
    current_sim_date = start_date + timedelta(days=day)
    print(f"\n\n>>> Day {day+1}: {current_sim_date.strftime('%Y-%m-%d')} <<<")
    
    # Run the daily cycle
    # Note: In a real backtest, we would mock the NewsAgent to return historical news.
    # Here, since we are using a live LLM search, it will fetch *current* news.
    # To make this a true "simulation" of logic (Forward Test), we accept that news is current,
    # but we simulate the passage of time for T+7 rules and price fluctuations.
    strategy.run_daily_cycle(current_sim_date)
    
    # Optional: Pause to avoid rate limits
    # import time
    # time.sleep(2)

print("\n=== Simulation Complete ===")
print("Final Inventory:")
print(my_inventory)